## In this Notebook we look at how the different models can interact with each other based on the context. We give openAI, Claude and Gemini models personas and ask them to talk about a topic like Global Warming. It is a fun way to learn about the different models and how they can interact with each other

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai

In [ ]:
# Load environment variables in a file called .env


load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')


In [ ]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

googleAI = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

## As a simple case prompts are organised into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

This structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

In [ ]:
# Using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = 'gemini-2.0-flash-exp'
topic_to_discuss = "Global Warming"

gpt_system = "You are a chatbot named mini who is in a 3-way conversation with Haiku and Flash which will be a debate and argument. \
Your view point is conservative.  It can be even be described as 'far-right'. \
your thinking can be traditional at times. You will be given a topic to discuss. On the 5th round of discussion thank everyone,let everyone know who made the most convincing argument other than yourself."

claude_system = "You are a chatbot named haiku who is in a 3-way conversation with Mini and Flash which will be a debate and argument. \
You have a modern take on the world.  It can be even be described as 'far-left'. \
your thinking can be quite a shock to the traditionalists at times. You will be given a topic to discuss. On the 5th round of discussion thank everyone,let everyone know who made the most convincing argument other than yourself."

gemini_system = "You are a chatbot named flash who is in a 3-way conversation with Mini and Haiku which will be a debate and argument. \
Your view point is balanced.  It can be even be described as 'centred'. \
your thinking can be measured and practical. You will be given a topic to discuss. On the 5th round of discussion thank everyone,let everyone know who made the most convincing argument other than yourself."

gpt_messages = ["Hi there I'm Mini!"]
claude_messages = ["Hi I'm Haiku"]
gemini_messages = [f"Hi, I'm Flash. Lets discuss about {topic_to_discuss}."]

In [ ]:
def call_gpt(system_prompt = gpt_system):
    messages = [{"role": "system", "content": system_prompt}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "assistant", "content": claude})
        messages.append({"role": "user", "content": gemini})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
def call_claude(system_prompt = claude_system):
    messages = []
    
    for gpt, claude_message, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
        messages.append({"role": "assistant", "content": gemini})
    messages.append({"role": "user", "content": gemini_messages[-1]})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    
    message = claude.messages.create(
        model=claude_model,
        system=system_prompt,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
def call_gemini(system_prompt = gemini_system):
    messages = [{"role": "system", "content": system_prompt}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "assistant", "content": gemini})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    messages.append({"role": "user", "content": claude_messages[-1]})
    completion = googleAI.chat.completions.create(
        model=gemini_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
print(f"Mini:\n{gpt_messages[0]}\n")
print(f"Haiku:\n{claude_messages[0]}\n")
print(f"Flash:\n{gemini_messages[0]}\n")


for i in range(5):

    gpt_next = call_gpt()
    print(f"Mini:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    claude_next = call_claude()
    print(f"Haiku:\n{claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    print(f"Flash:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)
